## The Functions of the Discrete Voter Model

This notebook implements the following $6$ subroutines for the **discrete voter model**:
1. `make_grid`: create a probabilistic grid: an $\mathbf{R}^n$ array of values, $\omega$, that all sum to $1$
2. `shift_weight`: shift weight in a grid in a reversible way
3. `expec_votes`: given a grid, a candidate, and a district description, output the expectation of votes that candidate received
4. `prob_votes`: given a grid, a candidate, a district description, and the observed vote outcomes, output the probability of seeing that outcome
5. `mcmc`: run a Markov Chain Monte Carlo method on a state space of grids
6. `hill_climb`: optimize the expectation or probability with gradient descent

### `make_grid`

In [ ]:
def make_grid(n_dim, granularity):
    """
    Create a probabilistic grid.
    
    n_dim (int): the number of dimensions of the grid
    granularity (float): the length of a single cell
    """
    